In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
syedanwarafridi_vehicle_sales_data_path = kagglehub.dataset_download('syedanwarafridi/vehicle-sales-data')

print('Data source import complete.')


# Vehicle Sales Data Cleaning

## Overview
This notebook focuses on **cleaning and logically restoring missing data** in a vehicle sales dataset.  
Instead of using random or statistical imputation, we relied on **VIN-based inference** to recover the most accurate values possible.

The main challenge was the **high number of missing values** in key columns such as:
- `make`
- `model`
- `body`
- `trim`
- `transmission`

## Approach & Missing Value Recovery

Our strategy focused on **logical restoration**:

- Created intermediate VIN-based features:  
  - **`vin_pre`** (first 3 chars)  
  - **`vin8`** (first 8 chars)  
  - **`vin_3_4`** (positions 3–5)  
  - **`vin_4_5`** (positions 4–6)  

- Used these VIN segments to **map missing values** based on the most frequent combinations for each make and year.

### Processed Columns

- **make:** 10,301 → 0 (fully restored)
- **model:** 10,399 → 69
- **trim:** 10,651 → 911
- **body:** 13,195 → 350
- **transmission:** 65,356 → 224

### Columns Left Unchanged

Columns like **color, interior, condition, mmr, seller, state, saledate, and odometer**  
were kept unchanged when **no reliable mapping** was available.

## Results

Below is the **before vs after** missing value reduction:

| Column        | Before   | After  |
|---------------|----------|--------|
| year          | 0        | 0      |
| make          | 10301    | 0      |
| model         | 10399    | 69     |
| trim          | 10651    | 911    |
| body          | 13195    | 350    |
| transmission  | 65356    | 224    |
| vin           | 0        | 0      |
| state         | 26       | 26     |
| condition     | 11794    | 11794  |
| odometer      | 94       | 94     |
| color         | 749      | 749    |
| interior      | 749      | 749    |
| seller        | 26       | 26     |
| mmr           | 12       | 12     |
| sellingprice  | 12       | 12     |
| saledate      | 38       | 38     |

## Key Takeaway

This cleaning process:
- **Significantly improved dataset usability** for modeling.  
- Preserved **data integrity by avoiding random imputation**.  
- Focused on **logical, VIN-based restoration techniques**.


## Vehicle Sales Data Cleaning – Logical Missing Value Handling

This notebook demonstrates a VIN-based strategy to recover missing values logically,
focusing on key columns: `make`, `model`, `trim`, `body`, and `transmission`.


# 1. Imports & Setup

In [ ]:
import pandas as pd
import numpy as np

# 2. Load Data

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-sales-data/car_prices.csv')
print(df.shape)
df.describe(include = 'O')

(558837, 16)


,make,model,trim,body,transmission,vin,state,color,interior,seller,saledate
count,548536,548438,548186,545642,493485,558833,558837,558088,558088,558837,558825
unique,96,973,1963,87,4,550297,64,46,17,14263,3766
top,Ford,Altima,Base,Sedan,automatic,automatic,fl,black,black,nissan-infiniti lt,Tue Feb 10 2015 01:30:00 GMT-0800 (PST)
freq,93554,19349,55817,199437,475915,22,82945,110970,244329,19693,5334


In [ ]:
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [ ]:
df['vin_3_4'] = df['vin'].str[3:5]
df['vin_pre'] = df['vin'].str[:3]
df["vin8"] = df['vin'].str[:8]
df['vin_4_5'] = df['vin'].str[3:6]

# 3. Data Cleaning & Preprocessing

## Initial Preprocessing

In [ ]:
to_lower = df.select_dtypes(include = 'object').columns
df[to_lower] = df[to_lower].apply(lambda x: x.str.strip().str.lower())
#cols to lower case

In [ ]:
shifted = df['vin'].str.len() != 17
df[shifted]
# the rows are shifted

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,vin_3_4,vin_pre,vin8,vin_4_5
408161,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj4fm201708,NaN,46.0,4802,silver,gray,NaN,13200.0,16500,om,aut,automati,oma
417835,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj2fm258506,NaN,1.0,9410,white,gray,NaN,13300.0,10500,om,aut,automati,oma
421289,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj3fm276741,NaN,46.0,1167,blue,black,NaN,13200.0,12700,om,aut,automati,oma
424161,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj2fm285365,NaN,1.0,2172,gray,black,NaN,14050.0,8250,om,aut,automati,oma
427040,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj0fm227318,NaN,41.0,14872,gray,black,NaN,13700.0,14300,om,aut,automati,oma
427043,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj6fm218641,NaN,49.0,12655,red,black,NaN,13850.0,14500,om,aut,automati,oma
434424,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj7fm223475,NaN,46.0,15719,blue,black,NaN,13650.0,13500,om,aut,automati,oma
444501,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj5fm297123,NaN,2.0,6388,white,black,NaN,13850.0,10700,om,aut,automati,oma
453794,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj5fm219943,NaN,44.0,16633,silver,black,NaN,13600.0,13600,om,aut,automati,oma
461597,2015,volkswagen,jetta,se pzev w/connectivity,navitgation,sedan,automatic,3vwd17aj9fm219766,NaN,44.0,11034,black,black,NaN,13900.0,13000,om,aut,automati,oma


In [ ]:
cols = ['body','transmission','vin','state','condition','odometer','color','interior','seller','mmr','sellingprice','saledate']
df.loc[shifted, cols] = df.loc[shifted, cols].shift(-1,axis = 1)

/tmp/ipykernel_36/4068934545.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['4802' '9410' '1167' '2172' '14872' '12655' '15719' '6388' '16633'
 '11034' '2711' '6864' '339' '18384' '9887' '9837' '20379' '20627' '721'
 '6158' '2817' '5705' '18561' '2846' '9562' '5001']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[shifted, cols] = df.loc[shifted, cols].shift(-1,axis = 1)
/tmp/ipykernel_36/4068934545.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['16500' '10500' '12700' '8250' '14300' '14500' '13500' '10700' '13600'
 '13000' '14000' '9800' '12900' '13500' '9900' '12900' '13500' '13500'
 '8500' '13400' '12200' '15250' '13100' '7500' '12100' '13600']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[shifted, cols] =

In [ ]:
shifted = df['vin'].str.len() != 17
df[shifted]
#no shifted rows

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,vin_3_4,vin_pre,vin8,vin_4_5


In [ ]:
df.isna().sum()
missing_before = df.isna().sum()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.loc[df['make'] == 'toyota', 'vin'].str[:3].value_counts()

vin
4t1    12962
5td     4410
jtd     4127
2t1     4067
jte     2885
2t3     2182
5yf     1868
5tf     1622
1nx     1308
4t4     1160
jtm     1003
4t3      684
3tm      376
5tb      376
jt3      347
jt2      288
jtn      129
5te       83
vnk       39
4ta       37
jt4       11
jt5        2
Name: count, dtype: int64

## Handling make Using VIN Prefix

In [ ]:
df['vin_pre'].head()
#first 3 index to detect the the make

0    5xy
1    5xy
2    wba
3    yv1
4    wba
Name: vin_pre, dtype: object

In [ ]:
pre_make = (df.dropna(subset=['make'])
    .groupby('vin_pre')['make']
    .apply(lambda x: x.value_counts().idxmax()))

In [ ]:
df.loc[df['make'].isna(), 'make'] = (
    df.loc[df['make'].isna(), 'vin'].str[:3].map(pre_make))


In [ ]:
df.loc[df['make'].isna(), ['vin', 'year']].value_counts()

vin                year
1geeh90y85u550143  2005    1
1jccf87exft014425  1985    1
wdype845675178050  2007    1
wdypd744x65871254  2006    1
wdypd644365880579  2006    1
wdapf3cc5d9552697  2013    1
wdapf3cc4d9554196  2013    1
wdapf3cc4d9552920  2013    1
wdapf3cc1d9553085  2013    1
4gldv13w37d203460  2007    1
4gldv13w27d199384  2007    1
3d6ws28d57g770072  2007    1
3d2ws28t69g526541  2009    1
2j4fy19e4lj510779  1990    1
2fdfp73933x203245  2003    1
1l1fm88w17y636036  2007    1
1l1fm81w63y651555  2003    1
1l1fm81w61y669258  2001    1
1l1fm81w4wy701356  1998    1
1l1fm81w1xy707715  1999    1
1l1fm81w13y674855  2003    1
wdype845975168001  2007    1
Name: count, dtype: int64

In [ ]:
df.loc[df['make'].isna(), 'vin'].str[:3].value_counts()

vin
1l1    6
wdy    4
wda    4
4gl    2
3d2    1
1jc    1
1ge    1
2j4    1
2fd    1
3d6    1
Name: count, dtype: int64

In [ ]:
df['make'].unique()

array(['kia', 'bmw', 'volvo', 'nissan', 'chevrolet', 'audi', 'ford',
       'hyundai', 'buick', 'cadillac', 'acura', 'lexus', 'infiniti',
       'jeep', 'mercedes-benz', 'mitsubishi', 'mazda', 'mini',
       'land rover', 'lincoln', 'jaguar', 'volkswagen', 'toyota',
       'subaru', 'scion', 'porsche', 'dodge', 'fiat', 'chrysler',
       'ferrari', 'honda', 'gmc', 'ram', 'smart', 'bentley', 'pontiac',
       'saturn', 'maserati', 'mercury', 'hummer', 'landrover', 'mercedes',
       'gmc truck', 'saab', 'suzuki', 'oldsmobile', 'isuzu', 'dodge tk',
       'geo', 'rolls-royce', 'mazda tk', 'hyundai tk', 'mercedes-b', 'vw',
       'daewoo', 'chev truck', 'ford tk', 'plymouth', 'ford truck',
       'tesla', 'airstream', 'dot', 'aston martin', 'fisker', nan,
       'lamborghini', 'lotus'], dtype=object)

In [ ]:
vin_make_map = {
    '1jc': 'jeep',
    '2j4': 'jeep',
    '1ge': 'cadillac',
    'wdy': 'freightliner',
    '4gl': 'buick',
    '3d2': 'dodge',
    '3d6': 'dodge',
    '2fd': 'ford',
    '1l1': 'lincoln',
    'wda': 'mercedes-benz'}

df.loc[df['make'].isna(), 'make'] = df.loc[df['make'].isna(), 'vin_pre'].map(vin_make_map)

In [ ]:
df['make'].isna().sum()
#make col is done

0

## Handling model, trim, body, transmission

In [ ]:
df['vin8'].sample(10)
#first 8 index to detect model, transmission, body, trim

304665    2c4rc1bg
332863    5n1aa0ne
462783    4a3aa46g
272776    1fm5k8d8
248667    1fmcu0d7
190180    jn1cv6ap
265849    jthbe1bl
558391    wddgf81x
369024    kmhct4ae
532771    4t1bf1fk
Name: vin8, dtype: object

In [ ]:
model = (df.dropna(subset=['model'])
               .drop_duplicates('vin8')
               .set_index('vin8')['model'])

df.loc[df['model'].isna() & df['vin8'].notna(), 'model'] = (
    df.loc[df['model'].isna() & df['vin8'].notna(), 'vin8'].map(model))

In [ ]:
df['model'].isna().sum()

746

In [ ]:
print(df['transmission'].value_counts())
print(df['transmission'].isna().sum())

transmission
automatic    475937
manual        17544
Name: count, dtype: int64
65356


In [ ]:
fill_table = df.dropna(subset=['trim','body','model','transmission']).groupby('vin_4_5')[['trim','body','model','transmission']].first()
for col in ['trim','body','model','transmission']:
    df.loc[df[col].isna(), col] = df.loc[df[col].isna(), 'vin_4_5'].map(fill_table[col])


In [ ]:
df.isna().sum()

year                0
make                0
model             141
trim              911
body             1109
transmission      228
vin                 0
state              26
condition       11794
odometer           94
color             749
interior          749
seller             26
mmr                12
sellingprice       12
saledate           38
vin_3_4             4
vin_pre             4
vin8                4
vin_4_5             4
dtype: int64

## Final Refinements & Manual Mapping

In [ ]:
manual_table = {
    'lm9': [
        ['base', 'coupe', 'mark viii', 'automatic']
    ],
    'ha7': [
        ['unlimited 70th anniversary', 'suv', 'wrangler', 'automatic']
    ],
    'zar': [
        ['le', 'suv', 'highlander', 'automatic']
    ],
    'gv2': [
        ['autobiography', 'suv', 'range rover', 'automatic']
    ],
    'pcc': [
        ['sx', 'suv', 'sportage', 'automatic'],
        ['ex', 'suv', 'sportage', 'automatic']
    ],
    'ks1': [
        ['slt', 'mega cab', 'ram pickup 1500', 'automatic'],
        ['laramie', 'mega cab', 'ram pickup 1500', 'automatic']
    ],
    'bu3': [
        ['3.0i', 'convertible', 'z4', 'automatic'],
        ['gxe', 'sedan', 'altima', 'automatic'],
        ['se', 'sedan', 'altima', 'automatic']
    ],
    'fd3': [
        ['lt', 'convertible', 'camaro', 'automatic']
    ],
    'vn3': [
        ['limited', 'suv', '4runner', 'automatic'],
        ['sr5 v6', 'suv', '4runner', 'automatic']
    ],
    'su4': [
        ['eddie bauer', 'suv', 'excursion', 'automatic'],
        ['limited', 'suv', 'excursion', 'automatic'],
        ['4x4 v8 limited', 'suv', 'excurs', 'automatic']
    ],
    'ng5': [
        ['gls', 'sedan', 'cutlass', 'automatic']
    ],
    'cv9': [
        ['s', 'suv', 'tiguan', 'automatic']
    ],
    'yy5': [
        ['platinum', 'suv', 'sequoia', 'automatic']
    ],
    'tt2': [
        ['laramie', 'crew cab', 'ram pickup 2500', 'automatic'],
        ['laramie longhorn edition', 'crew cab', '2500', 'automatic']
    ],
    '1b5': [
        ['ls', 'sedan', 'malibu', 'automatic']
    ],
    'br5': [
        ['base', 'sedan', 'continental flying spur', 'automatic']
    ],
    'fmb': [
        ['tdi premium plus quattro', 'sedan', 'a6', 'automatic']
    ],
    '1e5': [
        ['lt', 'sedan', 'malibu', 'automatic']
    ],
    'ag2': [
        ['base', 'suv', 'lr4', 'automatic'],
        ['v8 hse', 'suv', 'lr3', 'automatic'],
        ['hse', 'suv', 'lr4', 'automatic'],
        ['v8', 'suv', 'lr4', 'automatic']
    ],
    'rd5': [
        ['citadel', 'suv', 'durango', 'automatic']
    ]
}


In [ ]:
for vin_key, values in manual_table.items():
    for trim, body, model, trans in values:
        mask = df['vin_4_5'] == vin_key
        df.loc[mask, ['trim','body','model','transmission']] = [trim, body, model, trans]

In [ ]:
df.isna().sum()

year                0
make                0
model             141
trim              911
body             1109
transmission      228
vin                 0
state              26
condition       11794
odometer           94
color             749
interior          749
seller             26
mmr                12
sellingprice       12
saledate           38
vin_3_4             4
vin_pre             4
vin8                4
vin_4_5             4
dtype: int64

In [ ]:
fill_table = df.dropna(subset=['model','body']).groupby(['make','year','vin_3_4'])[['model','body']].first()
for col in ['model','body']:
    df.loc[df[col].isna(), col] = df.loc[df[col].isna()].apply(
        lambda x: fill_table.loc[(x['make'], x['year'], x['vin_3_4']), col]
                  if (x['make'], x['year'], x['vin_3_4']) in fill_table.index else x[col], axis=1 )

In [ ]:
df.isna().sum()

year                0
make                0
model              69
trim              911
body              350
transmission      228
vin                 0
state              26
condition       11794
odometer           94
color             749
interior          749
seller             26
mmr                12
sellingprice       12
saledate           38
vin_3_4             4
vin_pre             4
vin8                4
vin_4_5             4
dtype: int64

## Validation

In [ ]:
missing_after = df.isna().sum()

print(pd.DataFrame({'Before': missing_before, 'After': missing_after}))


              Before  After
year               0      0
make           10301      0
model          10399     69
trim           10651    911
body           13195    350
transmission   65356    228
vin                0      0
state             26     26
condition      11794  11794
odometer          94     94
color            749    749
interior         749    749
seller            26     26
mmr               12     12
sellingprice      12     12
saledate          38     38
vin_3_4            4      4
vin_pre            4      4
vin8               4      4
vin_4_5            4      4


---

## Conclusion & Insights

- **Key Findings**:  
  - e.g., "Brand X offers the highest median sale price",  
  - "Mileage negatively correlates with price",  
  - "Most vehicles sold within Y model years."

- **Recommendations / Next Steps**:  
  - "Explore predictive modeling for price estimation",  
  - "Segment analysis by region or seller type."

---

**Author:** [Misk Mohammed]  
**LinkedIn:** [www.linkedin.com/in/misk-mohammed-aba08a2b6]  
**Kaggle Notebook:** [https://www.kaggle.com/code/miskbadr/vehicle-sales-cleanning]
